In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [4]:
from services.SQLiteService import SQLiteService

database = SQLiteService()
database.connect()
data = database.select_data()

In [5]:
import pandas as pd
from datetime import datetime

# Conversion en DataFrame
df = pd.DataFrame(data)

# Conversion de la colonne datetime en type datetime
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%dT%H:%M')

# Définition de l'index temporel
df = df.set_index('datetime').sort_index()

# Vérification des types
print(df.dtypes)

temperature    float64
dtype: object


In [6]:
df.head()

,temperature
datetime,
2025-01-01 00:00:00,0.8
2025-01-01 01:00:00,2.1
2025-01-01 02:00:00,2.6
2025-01-01 03:00:00,3.1
2025-01-01 04:00:00,3.6
